# Chunking and SQLite

In this notebook, we'll see a couple of techniques that can be used when working with large file in Python.

In [2]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

Before we do anything else, let's see how many rows are contained in the Calls for Service file. 

In [ ]:
#using terminal  "wc -l file.txt"

In [3]:
npi_cols=['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
 'Provider Gender Code',
 'Healthcare Provider Taxonomy Code_1',
 'Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Taxonomy Code_2',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Taxonomy Code_3',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Taxonomy Code_4',
 'Healthcare Provider Primary Taxonomy Switch_4',
 'Healthcare Provider Taxonomy Code_5',
 'Healthcare Provider Primary Taxonomy Switch_5',
 'Healthcare Provider Taxonomy Code_6',
 'Healthcare Provider Primary Taxonomy Switch_6',
 'Healthcare Provider Taxonomy Code_7',
 'Healthcare Provider Primary Taxonomy Switch_7',
 'Healthcare Provider Taxonomy Code_8',
 'Healthcare Provider Primary Taxonomy Switch_8',
 'Healthcare Provider Taxonomy Code_9',
 'Healthcare Provider Primary Taxonomy Switch_9',
 'Healthcare Provider Taxonomy Code_10',
 'Healthcare Provider Primary Taxonomy Switch_10',
 'Healthcare Provider Taxonomy Code_11',
 'Healthcare Provider Primary Taxonomy Switch_11',
 'Healthcare Provider Taxonomy Code_12',
 'Healthcare Provider Primary Taxonomy Switch_12',
 'Healthcare Provider Taxonomy Code_13',
 'Healthcare Provider Primary Taxonomy Switch_13',
 'Healthcare Provider Taxonomy Code_14',
 'Healthcare Provider Primary Taxonomy Switch_14',
 'Healthcare Provider Taxonomy Code_15',
 'Healthcare Provider Primary Taxonomy Switch_15']

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [3]:
db = sqlite3.connect('nppes.sqlite')

Now, we can chunk through the data and for each row, add the rows to a table in our sqlite database. 
To keep track of how much progress has been made, we can use the `tqdm` library.

In [4]:
for chunk in tqdm(pd.read_csv('NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', usecols=npi_cols, chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_').replace('(', '').replace(')','').replace('*','') for x in chunk.columns]      # Clean up the column names
    conditions = [
    (chunk['healthcare_provider_primary_taxonomy_switch_1']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_2']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_3']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_4']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_5']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_6']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_7']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_8']=='Y')]

# create a list of the values we want to assign for each condition
    values = [chunk['healthcare_provider_taxonomy_code_1'],
          chunk['healthcare_provider_taxonomy_code_2'],
          chunk['healthcare_provider_taxonomy_code_3'],
          chunk['healthcare_provider_taxonomy_code_4'],
          chunk['healthcare_provider_taxonomy_code_5'],
          chunk['healthcare_provider_taxonomy_code_6'],
          chunk['healthcare_provider_taxonomy_code_7'],
          chunk['healthcare_provider_taxonomy_code_8']]

# create a new column and use np.select to assign values to it using our lists as arguments
    chunk['taxonomy'] = np.select(conditions, values)
    chunk=chunk[chunk.columns.drop(list(chunk.filter(regex='healthcare_provider_')))]
    chunk.to_sql('npi_data', db, if_exists = 'append', index = False)

#figure out this later
#def find_primary_code(row):
    #col = 'Healthcare Provider Primary Taxonomy Switch_'
    #for i in range(1, 16):
        #if row[col + str(i)] == 'Y':
            #return row['Healthcare Provider Taxonomy Code_' + str(i)]

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types.S

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (32,79,82,83,86,87,90,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (67,70,71,74,75,78,79,82,83,86,87,9

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
db.close()

In [4]:
db = sqlite3.connect('nppes.sqlite')
query='''
select *
from npi_data;
'''
npi_data = pd.read_sql(query, db)
npi_data

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,provider_gender_code,taxonomy
0,1679576722,1.0,NaN,None,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,6.88473e+08,M,207X00000X
1,1588667638,1.0,NaN,None,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,3.22045e+08,M,207RC0000X
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,2.83045e+08,None,251G00000X
3,1306849450,1.0,NaN,None,None,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,None,ATHENS,TX,7.57513e+08,M,2085R0202X
4,1215930367,1.0,NaN,None,None,GRESSOT,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,7.70901e+08,M,207RH0003X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6714033,1326630724,1.0,NaN,None,None,DELMONTE,ROXANA,None,None,None,MSN-APRN-FNP-C,8895 FONTAINEBLEAU BLVD APT 309,None,MIAMI,FL,3.31724e+08,F,363LF0000X
6714034,1912599325,1.0,NaN,None,None,KATEHIS,GIORGIOS,None,None,None,PHARMD,391 W MAIN ST,None,HUNTINGTON,NY,1.17433e+08,M,183500000X
6714035,1821680232,1.0,NaN,None,None,NGANDJO YEPNDJOUO,SARIETTE,None,MS.,None,None,750 S STATE ST,None,ELGIN,IL,6.01238e+08,F,163WP0808X
6714036,1730771148,1.0,NaN,None,None,JONES,JEANETTE,MARIE,None,None,None,412 1ST ST,None,MIDDLESEX,NJ,8.84615e+07,F,390200000X


In [ ]:
npi_data.info()

In [6]:
db = sqlite3.connect('nppes.sqlite')
nucc_tax=pd.read_csv('NPPES_Data_Dissemination_February_2021/nucc_taxonomy_210.csv')
nucc_tax.columns = [x.lower().replace(' ', '_') for x in nucc_tax.columns]
nucc_tax.to_sql('nucc_tax', db,  if_exists='replace', index = False)    


In [6]:
query='''
select *
from nucc_tax;
'''
nucc_tax = pd.read_sql(query, db)
nucc_tax

,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,10/1/2003,None,None,[7/1/2003: new],Multi-Specialty Group
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,10/1/2003,None,None,[7/1/2003: new],Single Specialty Group
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,4/1/2003,None,7/1/2007,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,4/1/2003,None,None,None,Allergy Physician
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,4/1/2003,None,None,None,Clinical & Laboratory Immunology (Allergy & Im...
...,...,...,...,...,...,...,...,...,...,...
860,343800000X,Transportation Services,Secured Medical Transport (VAN),None,A public or privately owned transportation ser...,4/1/2002,None,None,None,Secured Medical Transport (VAN)
861,344600000X,Transportation Services,Taxi,None,A land commercial vehicle used for the transpo...,4/1/2002,None,None,None,Taxi
862,347D00000X,Transportation Services,Train,None,An organization or business licensed to provid...,4/1/2002,None,None,None,Train
863,347E00000X,Transportation Services,Transportation Broker,None,An organization that provides transportation f...,4/1/2002,None,1/1/2021,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker


In [ ]:
db.close()

In [ ]:
nucc_tax.info()

To speed up queries which use a specific column, we can create an **index** on that column. This causes the database to store that column in a way that helps it to retrieve rows quicker.

In [ ]:
db = sqlite3.connect('nppes.sqlite')
db.execute('CREATE INDEX taxonomy ON npi_data(taxonomy)')
db.execute('CREATE INDEX code ON nucc_tax(code)')

In [23]:
db = sqlite3.connect('nppes.sqlite')
cbsa=pd.read_csv('NPPES_Data_Dissemination_February_2021/ZIP_CBSA_122020.csv')
cbsa.columns = [x.lower().replace(' ', '_') for x in cbsa.columns]
cbsa.to_sql('cbsa', db,  if_exists='replace', index = False)

In [7]:
db = sqlite3.connect('nppes.sqlite')
query= '''
select *       
from cbsa 
'''
cbsa = pd.read_sql(query, db)
cbsa.head(25)

,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,501,35620,0.000000,1.000000,0.000000,1.000000
1,601,38660,1.000000,1.000000,1.000000,1.000000
2,602,10380,1.000000,1.000000,1.000000,1.000000
3,603,10380,1.000000,1.000000,1.000000,1.000000
4,604,10380,1.000000,1.000000,1.000000,1.000000
5,605,10380,1.000000,1.000000,1.000000,1.000000
6,606,99999,1.000000,1.000000,1.000000,1.000000
7,610,10380,1.000000,1.000000,1.000000,1.000000
8,611,10380,1.000000,1.000000,1.000000,1.000000
9,612,11640,0.995485,0.999191,0.996909,0.995867


In [10]:
db = sqlite3.connect('nppes.sqlite')
query= '''
select *      
from npi_data as np
left join nucc_tax as nu
on np.taxonomy=nu.code 
join cbsa as c
on np.provider_business_practice_location_address_postal_code=c.zip 
'''
npp_nucc_tax = pd.read_sql(query, db)
npp_nucc_tax.head(25)

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,deactivation_date,last_modified_date,notes,display_name,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1396748448,1.0,NaN,None,None,YOUNG-MAYKA,CYNTHIA,J,MS.,None,...,None,None,None,Physician Assistant,13069,45060,1.000000,1.000000,1.000000,1.000000
1,1669475612,1.0,NaN,None,None,TROTCHIE,DEBBIE,C,None,None,...,None,None,None,Women's Health Nurse Practitioner,89148,29820,1.000000,1.000000,1.000000,1.000000
2,1043213093,2.0,NaN,<UNAVAIL>,HOME I V SPECIALISTS INC,None,None,None,None,None,...,None,None,Source: National Home Infusion Association [1...,Home Infusion Therapy Pharmacy,72401,27860,0.991766,0.999716,1.000000,0.993251
3,1043213093,2.0,NaN,<UNAVAIL>,HOME I V SPECIALISTS INC,None,None,None,None,None,...,None,None,Source: National Home Infusion Association [1...,Home Infusion Therapy Pharmacy,72401,37500,0.008234,0.000284,0.000000,0.006749
4,1194728139,1.0,NaN,None,None,STEWART,SUSAN,S,MRS.,None,...,None,None,None,Medical Physician Assistant,96793,27980,1.000000,1.000000,1.000000,1.000000
5,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,24620,0.334719,0.400000,0.361111,0.338293
6,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,28700,0.646392,0.600000,0.638889,0.643996
7,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,34100,0.018889,0.000000,0.000000,0.017712
8,1255334280,1.0,NaN,None,None,HENSARLING,JAMES,KENNETH,DR.,None,...,None,7/1/2011,"Source: American Board of Medical Specialties,...",Rheumatology Physician,39232,27140,1.000000,1.000000,1.000000,1.000000
9,1962405019,1.0,NaN,None,None,HAMAMI,ANWAR,None,None,None,...,None,7/1/2011,"Source: American Board of Medical Specialties,...",Emergency Medicine Physician,44236,10420,1.000000,1.000000,1.000000,1.000000


In [11]:
df=npp_nucc_tax[['npi', 'provider_business_practice_location_address_postal_code', 'specialization', 'zip','cbsa']]
df.sort_values(by=['cbsa'])

,npi,provider_business_practice_location_address_postal_code,specialization,zip,cbsa
188345,1013097880,57401.0,None,57401,10100
183907,1811085970,57401.0,None,57401,10100
737433,1851934335,57428.0,None,57428,10100
177283,1942395835,57428.0,Rural Health,57428,10100
252338,1013062280,57428.0,Internist,57428,10100
...,...,...,...,...,...
291016,1295857225,41240.0,None,41240,99999
291021,1023130069,59865.0,None,59865,99999
291023,1134241177,74301.0,Addiction (Substance Use Disorder),74301,99999
291055,1548382500,40489.0,Student Health,40489,99999


In [ ]:
cbsa.info()

In [ ]:
df.duplicated(subset=['npi'], ).value_counts()

In [8]:
npp_nucc=pd.merge(npi_data, nucc_tax,left_on='taxonomy', right_on='code', how='left')

In [9]:
npp_nucc.head(50)


,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
0,1679576722,1.0,NaN,None,None,WIEBE,DAVID,A,None,None,...,207X00000X,Allopathic & Osteopathic Physicians,Orthopaedic Surgery,None,An orthopaedic surgeon is trained in the prese...,4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Orthopaedic Surgery Physician
1,1588667638,1.0,NaN,None,None,PILCHER,WILLIAM,C,DR.,None,...,207RC0000X,Allopathic & Osteopathic Physicians,Internal Medicine,Cardiovascular Disease,An internist who specializes in diseases of th...,4/1/2003,None,7/1/2011,Source: American Osteopathic Board of Internal...,Cardiovascular Disease Physician
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,...,251G00000X,Agencies,"Hospice Care, Community Based",None,Definition to come...,4/1/2002,None,None,None,Community Based Hospice Care Agency
3,1306849450,1.0,NaN,None,None,SMITSON,HAROLD,LEROY,DR.,II,...,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...",4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician
4,1215930367,1.0,NaN,None,None,GRESSOT,LAURENT,None,DR.,None,...,207RH0003X,Allopathic & Osteopathic Physicians,Internal Medicine,Hematology & Oncology,An internist doctor of osteopathy that special...,4/1/2003,None,7/1/2011,Source: American Osteopathic Board of Internal...,Hematology & Oncology Physician
5,1023011178,2.0,NaN,<UNAVAIL>,COLLABRIA CARE,None,None,None,None,None,...,251G00000X,Agencies,"Hospice Care, Community Based",None,Definition to come...,4/1/2002,None,None,None,Community Based Hospice Care Agency
6,1932102084,1.0,NaN,None,None,ADUSUMILLI,RAVI,K,None,None,...,207RC0000X,Allopathic & Osteopathic Physicians,Internal Medicine,Cardiovascular Disease,An internist who specializes in diseases of th...,4/1/2003,None,7/1/2011,Source: American Osteopathic Board of Internal...,Cardiovascular Disease Physician
7,1841293990,1.0,NaN,None,None,WORTSMAN,SUSAN,None,None,None,...,231H00000X,"Speech, Language and Hearing Service Providers",Audiologist,None,"(1) A specialist in evaluation, habilitation a...",4/1/2002,None,None,Source: (1) American Speech-Language-Hearing A...,Audiologist
8,1750384806,1.0,NaN,None,None,BISBEE,ROBERT,None,DR.,None,...,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,None,"A physician who provides long-term, comprehens...",4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Internal Medicine Physician
9,1669475711,1.0,NaN,None,None,SUNG,BIN,SHENG,None,None,...,208000000X,Allopathic & Osteopathic Physicians,Pediatrics,None,"A pediatrician is concerned with the physical,...",4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Pediatrics Physician


In [10]:
npp_nucc['postal_code']=npp_nucc['provider_business_practice_location_address_postal_code'].astype(str).str[0:5]

cbsa['zip1']=cbsa['zip'].astype(str).str[0:5]


In [11]:
npp_nucc_cbsa=pd.merge(npp_nucc, cbsa,left_on= 'postal_code', right_on='zip1', how='left')

In [25]:
npp_nucc_cbsa_nash=npp_nucc_cbsa[npp_nucc_cbsa['cbsa']==34980]


In [14]:
npp_nucc_cbsa_nash.to_csv('npp_taxo_cbsa_nash.csv',index=False)

In [ ]:
npp_nucc_cbsa_nash.duplicated(subset=['npi']).value_counts()

In [12]:
npp_nucc_cbsa[['npi','specialization','zip']].head(50)

,npi,specialization,zip
0,1679576722,None,68847.0
1,1588667638,Cardiovascular Disease,32204.0
2,1497758544,None,28304.0
3,1306849450,Diagnostic Radiology,75751.0
4,1306849450,Diagnostic Radiology,75751.0
5,1215930367,Hematology & Oncology,77090.0
6,1023011178,None,94559.0
7,1932102084,Cardiovascular Disease,43615.0
8,1841293990,None,10010.0
9,1750384806,None,79415.0


In [ ]:
pd.read_csv('DocGraph_Hop_Teaming_2017_Non_Commercial/DocGraph_Hop_Teaming_2017.csv', nrows = 1000).info()

In [16]:
db = sqlite3.connect('nppes.sqlite')
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2017_Non_Commercial/DocGraph_Hop_Teaming_2017.csv', chunksize = 10000)):
    chunk[(chunk['transaction_count']>=50) & (chunk['average_day_wait']<=50)]
    chunk.to_sql('hop', db, if_exists = 'replace', index = False)


In [24]:
db = sqlite3.connect('nppes.sqlite')
query='''
select *
from hop;
'''
hop_team = pd.read_sql(query, db)
hop_team 

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1265698203,1255364378,99,102,69.255,75.830
1,1235168139,1255364378,12,14,30.643,29.319
2,1235160482,1255364378,22,22,81.091,67.251
3,1235132549,1255364378,16,16,90.500,89.803
4,1235127929,1255364378,11,13,37.692,45.494
...,...,...,...,...,...,...
902,1235291360,1255367132,12,13,44.769,96.969
903,1235180266,1255367132,23,23,13.739,21.020
904,1265448617,1255367132,15,16,10.938,25.702
905,1265437644,1255367132,78,82,26.561,51.304


In [23]:
npp_nucc_cbsa_nash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41181 entries, 54 to 7402091
Data columns (total 37 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   npi                                                      41181 non-null  int64  
 1   entity_type_code                                         41181 non-null  float64
 2   replacement_npi                                          0 non-null      float64
 3   employer_identification_number_ein                       8308 non-null   object 
 4   provider_organization_name_legal_business_name           8308 non-null   object 
 5   provider_last_name_legal_name                            32872 non-null  object 
 6   provider_first_name                                      32873 non-null  object 
 7   provider_middle_name                                     21229 non-null  object 
 8   provider_name_prefix_te

In [18]:
npp_nucc_cbsa_nash['npp_npi']=npp_nucc_cbsa_nash['npi'].astype(str)
hop_team['hop_from_npi']=hop_team['from_npi'].astype(str)
hop_team['hop_to_npi']=hop_team['to_npi'].astype(str)

<ipython-input-18-891ac980e93c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npp_nucc_cbsa_nash['npp_npi']=npp_nucc_cbsa_nash['npi'].astype(str)


In [28]:
npp_nucc_cbsa_nash_hop=pd.merge(npp_nucc_cbsa_nash, hop_team, left_on= 'npi', right_on='from_npi', how='inner')

In [31]:
npp_nucc_cbsa_nash_hop1=pd.merge(npp_nucc_cbsa_nash, hop_team, left_on= 'npi', right_on='to_npi', how='inner')

In [ ]:
npp_nucc[['npi','specialization']]

In [33]:
npp_nucc_cbsa_nash_hop1.head()

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,bus_ratio,oth_ratio,tot_ratio,zip1,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1255365284,2.0,NaN,<UNAVAIL>,DANIEL K BREGMAN MD PC,None,None,None,None,None,...,1.0,1.0,1.0,37203,1235186800,1255365284,76,86,66.860,60.453
1,1255365284,2.0,NaN,<UNAVAIL>,DANIEL K BREGMAN MD PC,None,None,None,None,None,...,1.0,1.0,1.0,37203,1235256512,1255365284,21,27,35.556,44.874
2,1255365284,2.0,NaN,<UNAVAIL>,DANIEL K BREGMAN MD PC,None,None,None,None,None,...,1.0,1.0,1.0,37203,1225135551,1255365284,19,20,84.550,64.784
3,1255365284,2.0,NaN,<UNAVAIL>,DANIEL K BREGMAN MD PC,None,None,None,None,None,...,1.0,1.0,1.0,37203,1225119183,1255365284,14,21,11.429,27.757
4,1255365284,2.0,NaN,<UNAVAIL>,DANIEL K BREGMAN MD PC,None,None,None,None,None,...,1.0,1.0,1.0,37203,1255646907,1255365284,22,30,26.533,26.679


In [34]:
npp_hop_dfs=[npp_nucc_cbsa_nash_hop,npp_nucc_cbsa_nash_hop1]

In [35]:
npp_nucc_cbsa_nash_hop_all=pd.concat(npp_hop_dfs, keys=['from', 'to'])

In [36]:
npp_nucc_cbsa_nash_hop_all

npi  entity_type_code  replacement_npi  \
from 0   1235124207               1.0              NaN   
     1   1245221092               2.0              NaN   
     2   1245221092               2.0              NaN   
     3   1245221092               2.0              NaN   
     4   1235186800               2.0              NaN   
     5   1235186800               2.0              NaN   
     6   1235186800               2.0              NaN   
     7   1235186800               2.0              NaN   
     8   1235186800               2.0              NaN   
     9   1265475156               1.0              NaN   
     10  1265445506               2.0              NaN   
     11  1265445506               2.0              NaN   
     12  1245348580               1.0              NaN   
     13  1225135551               2.0              NaN   
     14  1255419255               2.0              NaN   
     15  1245393057               2.0              NaN   
     16  1255513388               2.0              NaN   
     17  1255646907               1.0              NaN   
     18  1245503135               1.0              NaN   
to   0   1255365284               2.0              NaN   
     1   1255365284               2.0              NaN   
     2   1255365284               2.0              NaN   
     3   1255365284               2.0              NaN   
     4   1255365284               2.0              NaN   
     5   1255365284               2.0              NaN   
     6   1255365284               2.0              NaN   
     7   1255365284               2.0              NaN   
     8   1255365284               2.0              NaN   
     9   1255365284               2.0              NaN   
     10  1255365284               2.0              NaN   
     11  1255365284               2.0              NaN   
     12  1255365284               2.0              NaN   
     13  1255365987               2.0              NaN   

        employer_identification_number_ein  \
from 0                                None   
     1                           <UNAVAIL>   
     2                           <UNAVAIL>   
     3                           <UNAVAIL>   
     4                           <UNAVAIL>   
     5                           <UNAVAIL>   
     6                           <UNAVAIL>   
     7                           <UNAVAIL>   
     8                           <UNAVAIL>   
     9                                None   
     10                          <UNAVAIL>   
     11                          <UNAVAIL>   
     12                               None   
     13                          <UNAVAIL>   
     14                          <UNAVAIL>   
     15                          <UNAVAIL>   
     16                          <UNAVAIL>   
     17                               None   
     18                               None   
to   0                           <UNAVAIL>   
     1                           <UNAVAIL>   
     2                           <UNAVAIL>   
     3                           <UNAVAIL>   
     4                           <UNAVAIL>   
     5                           <UNAVAIL>   
     6                           <UNAVAIL>   
     7                           <UNAVAIL>   
     8                           <UNAVAIL>   
     9                           <UNAVAIL>   
     10                          <UNAVAIL>   
     11                          <UNAVAIL>   
     12                          <UNAVAIL>   
     13                          <UNAVAIL>   

            provider_organization_name_legal_business_name  \
from 0                                                None   
     1                               VERUS HEALTHCARE, LLC   
     2                               VERUS HEALTHCARE, LLC   
     3                               VERUS HEALTHCARE, LLC   
     4                                 PATHGROUP LABS, LLC   
     5                                 PATHGROUP LABS, LLC   
     6                                 PATHG